In [15]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import json
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, LeaveOneGroupOut
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay
import xgboost as xgb
import warnings
import os

In [16]:
def evaluator(y_pred, y_test, verbose=False):
    """Returns evaluation metric scores"""
    accuracy = accuracy_score(y_pred=y_pred, y_true=y_test)
    balanced_accuracy = balanced_accuracy_score(y_pred=y_pred, y_true=y_test)
    f1 = f1_score(y_pred=y_pred, y_true=y_test, average='weighted')
    recall = recall_score(y_pred=y_pred, y_true=y_test, average='weighted')
    precision = precision_score(y_pred=y_pred, y_true=y_test, average='weighted')
    confusion = confusion_matrix(y_pred=y_pred, y_true=y_test)

    # display scores
    if verbose:
        ConfusionMatrixDisplay(confusion_matrix=confusion, display_labels=[False, True]).plot(cmap=plt.cm.Blues)
        plt.title('Physical fatigue')

        print(f'accuracy: {accuracy}\n'
              f'balanced accuracy: {balanced_accuracy}\n'
              f'f1 (weighted): {f1}\n'
              f'recall (weighted): {recall}\n'
              f'precision (weighted): {precision}')

    return {'accuracy': accuracy,
            'balanced_accuracy': balanced_accuracy,
            'f1': f1,
            'recall': recall,
            'precision': precision,
            'confusion': confusion}

In [17]:
VARIABLES = ['ActivityCounts', 'Barometer', 'BloodPerfusion',
             'BloodPulseWave', 'EnergyExpenditure', 'GalvanicSkinResponse', 'HR',
             'HRV', 'RESP', 'Steps', 'SkinTemperature', 'ActivityClass']

In [18]:
NORMALIZE_TRAIN = True # whether to normalize data acc. to training data
SHUFFLE = True # whether to shuffle data before applying CV

In [19]:
# for reproducability
SEED = 42

# Import data

In [20]:
# file path to data folder
path = './Output'

In [21]:
N, FEATURES = sum([1 for p in os.listdir(path) if p[:19] == 'feature_vector_stat']), \
              *np.load(path + '/feature_vector_stat0.npy').shape
print(f'datapoints: {N}, features: {FEATURES}')

datapoints: 317, features: 284


Feature vector, labels

In [22]:
# init
X = np.empty((N, FEATURES))
y = np.empty((N, 2))

# load individual datapoints
for i in range(N):
    X[i, ] = np.load(path + f'/feature_vector_stat{i}.npy', allow_pickle=True)
    y[i, ] = np.load(path + f'/labels_stat{i}.npy', allow_pickle=True)

Metadata (subjectID etc.)

In [23]:
with open(path + '/metadata_stat.txt') as f:
    metadata = f.read()

metadata = json.loads(metadata.replace('\'', '\"').replace('False', 'false').replace('True', 'true')) # doesn't accept other chars

# XGBoost

### 5-fold stratified CV

In [24]:
# separate label prediction
y_phf, y_mf = y[:, 0], y[:, 1]

In [27]:
%%time
# nested CV
folds = 5

with warnings.catch_warnings():
    # ignore sklearn warning
    warnings.filterwarnings('ignore')

    for fatigue in ('Physical fatigue', 'Mental fatigue'):
        # load labels
        print(f'Starting cross-validation for {fatigue}')
        y_ = {'Physical fatigue': y_phf, 'Mental fatigue': y_mf}[fatigue] # pick phF or MF

        # CV: performance evaluation
        cv = StratifiedKFold(n_splits=folds, shuffle=True, random_state=SEED) if SHUFFLE \
            else StratifiedKFold(n_splits=folds)
        scores_cv = []
        with tqdm(total=folds) as pbar:
            for i, (train_outer_index, test_outer_index) in enumerate(cv.split(X, y_)):
                # train/test split
                X_train, X_test = X[train_outer_index], X[test_outer_index]
                y_train, y_test = y_[train_outer_index], y_[test_outer_index]

                # normalize features (acc.to train set)
                if NORMALIZE_TRAIN:
                    scaler = StandardScaler()
                    scaler.fit(X_train)
                    X_train = scaler.transform(X_train, copy=True)
                    X_test = scaler.transform(X_test, copy=True)

                # model
                model = xgb.XGBClassifier(random_state=SEED, verbosity=0)

                # training
                model.fit(X_train, y_train)

                # evaluate
                y_pred = model.predict(X_test)
                scores = evaluator(y_pred, y_test, verbose=False)
                scores_cv.append(scores)

                # for progress bar
                pbar.update(1)
                pbar.set_description(f' Fold {i+1} F1: {scores["f1"]}')

        # final evaluation
        print('Performance model:')
        metrics = scores_cv[0].keys()
        for metric in metrics:
            # ignore confusion_matrix
            if metric == 'confusion':
                continue
            mean = np.mean([scores_cv_i[metric] for scores_cv_i in scores_cv])
            std = np.std([scores_cv_i[metric] for scores_cv_i in scores_cv])
            print(f' {metric}: {round(mean, 3)} +- {round(std, 3)} \n')

Starting cross-validation for Physical fatigue


 Fold 5 F1: 0.7142857142857143: 100%|██████████| 5/5 [00:00<00:00,  7.55it/s]


Performance model:
 accuracy: 0.785 +- 0.053 

 balanced_accuracy: 0.65 +- 0.058 

 f1: 0.768 +- 0.05 

 recall: 0.785 +- 0.053 

 precision: 0.778 +- 0.054 

Starting cross-validation for Mental fatigue


 Fold 5 F1: 0.6880952380952382: 100%|██████████| 5/5 [00:00<00:00,  6.07it/s]

Performance model:
 accuracy: 0.726 +- 0.046 

 balanced_accuracy: 0.619 +- 0.072 

 f1: 0.697 +- 0.061 

 recall: 0.726 +- 0.046 

 precision: 0.7 +- 0.059 

CPU times: total: 10.2 s
Wall time: 1.49 s


### Leave-one-subject-out (LOSO)

In [28]:
subjects = [meta['subjectID'] for meta in metadata]
print(f'Subjects: {np.unique(subjects)}')
print(f'Total subjects: {len(np.unique(subjects))}')

Subjects: [ 3  4  5  7  9 10 12 13 14 15 16 17 19 20 21 22 23 24 25 26 27]
Total subjects: 21


In [29]:
%%time
# nested CV
groups = subjects
folds = len(np.unique(subjects))

with warnings.catch_warnings():
    # ignore sklearn warning
    warnings.filterwarnings('ignore')

    for fatigue in ('Physical fatigue', 'Mental fatigue'):
        # load labels
        print(f'Starting cross-validation for {fatigue}')
        y_ = {'Physical fatigue': y_phf, 'Mental fatigue': y_mf}[fatigue] # pick phF or MF

        # CV: performance evaluation
        cv = LeaveOneGroupOut()
        scores_cv = []
        with tqdm(total=folds) as pbar:
            for i, (train_outer_index, test_outer_index) in enumerate(cv.split(X, y_, groups)):
                # train/test split
                X_train, X_test = X[train_outer_index], X[test_outer_index]
                y_train, y_test = y_[train_outer_index], y_[test_outer_index]

                # normalize features (acc.to train set)
                if NORMALIZE_TRAIN:
                    scaler = StandardScaler()
                    scaler.fit(X_train)
                    X_train = scaler.transform(X_train, copy=True)
                    X_test = scaler.transform(X_test, copy=True)

                # model
                model = xgb.XGBClassifier(random_state=SEED, verbosity=0)

                # training
                model.fit(X_train, y_train)

                # evaluate
                y_pred = model.predict(X_test)
                scores = evaluator(y_pred, y_test, verbose=False)
                scores_cv.append(scores)

                # for progress bar
                pbar.update(1)
                pbar.set_description(f' Fold {i+1} F1: {scores["f1"]}')

        # final evaluation
        print('Performance model:')
        metrics = scores_cv[0].keys()
        for metric in metrics:
            # ignore confusion_matrix
            if metric == 'confusion':
                continue
            mean = np.mean([scores_cv_i[metric] for scores_cv_i in scores_cv])
            std = np.std([scores_cv_i[metric] for scores_cv_i in scores_cv])
            print(f' {metric}: {round(mean, 3)} +- {round(std, 3)} \n')

Starting cross-validation for Physical fatigue


 Fold 21 F1: 0.3409090909090909: 100%|██████████| 21/21 [00:03<00:00,  6.91it/s] 


Performance model:
 accuracy: 0.617 +- 0.313 

 balanced_accuracy: 0.557 +- 0.298 

 f1: 0.598 +- 0.327 

 recall: 0.617 +- 0.313 

 precision: 0.625 +- 0.352 

Starting cross-validation for Mental fatigue


 Fold 21 F1: 0.7333333333333334: 100%|██████████| 21/21 [00:03<00:00,  6.32it/s]

Performance model:
 accuracy: 0.617 +- 0.26 

 balanced_accuracy: 0.544 +- 0.271 

 f1: 0.616 +- 0.262 

 recall: 0.617 +- 0.26 

 precision: 0.663 +- 0.291 

CPU times: total: 50.3 s
Wall time: 6.37 s
